<a href="https://colab.research.google.com/github/KyPython/LLMs-Can-t-Please-Them-All/blob/main/LLMs_Can't_Please_Them_All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPU Test

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Feb 17 16:47:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             43W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

GPU Test

In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.device_count()) # Should print 1 or more
print(torch.cuda.get_device_name(0))  # Should print your GPU name (Tesla T4)

True
1
NVIDIA A100-SXM4-40GB


Installs & Imports

In [ ]:
# Installs first to avoid conflicts
%pip install requests
%pip install transformers --upgrade
%pip install pytest
%pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install --upgrade pip setuptools wheel
%pip install mistral_inference
%pip install sentencepiece
%pip install scikit-learn
%pip uninstall sentence-transformers huggingface_hub -y
%pip install sentence-transformers huggingface_hub --upgrade --no-cache-dir
%pip install --upgrade transformers


# Now imports
import sys
import os
import re
import json
import requests
import time
import datetime
import logging
import logging.handlers
import argparse
import configparser
import subprocess
import torch.optim as optim
import numpy as np
import random
import time
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import logging
from torch.cuda.amp import autocast
import os
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity


# These imports should come after the installations and basic imports
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM # Importing AutoModelForSeq2SeqLM
# Explicitly import AutoModelForSeq2SeqLM
from transformers import AutoModelForSeq2SeqLM
from huggingface_hub import snapshot_download

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached mistral_inference-1.5.0-py3-none-any.whl.metadata (14 kB)
  Using cached mistral_common-1.5.3-py3-none-any.whl.metadata (4.5 kB)
Using cached mistral_inference-1.5.0-py3-none-any.whl (30 kB)
Using cached mistral_common-1.5.3-py3-none-any.whl (6.5 MB)
Found existing installation: sentence-transformers 3.4.1
Uninstalling sentence-transformers-3.4.1:
  Successfully uninstalled sentence-transformers-3.4.1
Found existing installation: huggingface-hub 0.28.1
Uninstalling huggingface-hub-0.28.1:
  Successfully uninstalled huggingface-hub-0.28.1


Load Model Function

In [ ]:
# Define device globally before defining the function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_model(model_name, device=device, low_cpu_mem_usage=False):
    """Loads a pre-trained language model and its tokenizer."""
    logging.info(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_class = AutoModelForSeq2SeqLM if "t5" in model_name.lower() else AutoModelForCausalLM

    # Load with low_cpu_mem_usage if specified
    if low_cpu_mem_usage:
        model = model_class.from_pretrained(model_name, low_cpu_mem_usage=True).to(device)
    else:
        model = model_class.from_pretrained(model_name).to(device)


    # Set padding token for GPT-Neo and GPT2
    if "gpt-neo" in model_name.lower() or "gpt2" in model_name.lower():
        tokenizer.pad_token = tokenizer.eos_token  # or '[PAD]' if you prefer

    # Use the appropriate pipeline based on model type:
    if "t5" in model_name.lower():
        return pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=device), tokenizer
    else:
        return pipeline("text-generation", model=model, tokenizer=tokenizer, device=device), tokenizer

Generate Essays Function

In [ ]:
def generate_and_save_essays(topics, model_name, pipeline_function, tokenizer):
    """Generates essays for each topic and saves them to separate files."""
    generated_essays = []

    model_params = {
        "gpt-neo": {"temperature": 1.2, "top_k": 50, "top_p": 0.9, "max_length": 150},
        "t5": {"temperature": 0.8, "top_k": 40, "top_p": 0.85, "max_length": 120},
        "gpt2": {"temperature": 1.5, "top_k": 60, "top_p": 0.95, "max_length": 130},
    }

    params = model_params.get(model_name, {"temperature": 1.0, "top_k": 50, "top_p": 0.9, "max_length": 100})

    for topic_id, topic in topics:
        logging.info(f"Generating essay for topic ID: {topic_id} using model: {model_name}")

        try:
            # Ensure input is a string for all models
            input_text = topic if isinstance(topic, str) else topic[0]  # Handle list or string input

            # For T5, wrap the input_text in a list, but also make sure the pipeline is called correctly
            if "t5" in model_name.lower():
                logging.info("Calling pipeline for T5 model")
                result = pipeline_function([input_text],  # Pass input_text as a list for T5
                                           max_length=params["max_length"],
                                           temperature=params["temperature"],
                                           top_k=params["top_k"],
                                           top_p=params["top_p"],
                                           do_sample=True,
                                           num_return_sequences=1,
                                           truncation=True)

                # Check result structure and extract the essay
                logging.info(f"T5 result for topic ID {topic_id}: {result}")
                if isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict):
                    essay = result[0].get('generated_text', 'No text generated')
                else:
                    essay = 'Error in result format or empty response.'
            else:  # For GPT-Neo and GPT2
                logging.info("Calling pipeline for GPT-Neo/GPT2 model")
                result = pipeline_function(text_inputs=input_text,  # Pass input_text as 'text_inputs'
                                           max_length=params["max_length"],
                                           temperature=params["temperature"],
                                           top_k=params["top_k"],
                                           top_p=params["top_p"],
                                           do_sample=True,
                                           num_return_sequences=1,
                                           truncation=True)

                # Process result for GPT-Neo and GPT2
                essay = result[0]['generated_text'] if isinstance(result[0], dict) else result[0]

            filename = f"essay_{model_name}_{topic_id}.txt"
            with open(filename, "w") as f:
                f.write(essay)
            logging.info(f"Essay saved to: {filename}")
            generated_essays.append((topic_id, essay))

        except Exception as e:
            logging.error(f"Error generating essay for topic ID: {topic_id}, model: {model_name}. Error: {str(e)}")
            continue

    return generated_essays


Logging Setup

In [ ]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug("Logging is initialized")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"Using device: {device}")

if device.type == 'cuda':
    torch.cuda.set_device(0)  # Example: Set to GPU 0 if available

def flush_logs():
    for handler in logging.getLogger().handlers:
        handler.flush()

CUDA Check

In [ ]:
if torch.cuda.is_available():
    current_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f"CUDA memory allocated: {torch.cuda.memory_allocated(current_device)} bytes")
    logging.info(f"CUDA memory cached: {torch.cuda.memory_reserved(current_device)} bytes")
else:
    logging.info("CUDA is not available. Skipping memory logging.")

# Function to determine device and handle CUDA out of memory
def get_device():
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        if device.type == 'cuda':
            torch.zeros(1).to(device)  # Force device allocation to check if CUDA is working
        logging.info(f"Using device: {device}")
        return device
    except RuntimeError as e:
        if "out of memory" in str(e):
            logging.warning("CUDA out of memory, switching to CPU.")
            return torch.device('cpu')
        else:
            logging.error(f"Runtime error: {e}")
            raise e

Generate Essays

In [ ]:
topics = [
    ("1097671", "Compare and contrast the importance of self-reliance and adaptability in healthcare."),
    ("1726150", "Evaluate the effectiveness of management consulting in addressing conflicts within marketing."),
    ("3211968", "Discuss the role of self-reliance in achieving success in software engineering.")
]

essay_generation_models = {
    "gpt-neo": load_model("EleutherAI/gpt-neo-1.3B", device=device),  # Changed to GPT-Neo 1.3B
    "t5": load_model("google/flan-t5-large", device=device),  # Try smaller models like 'distil-t5'
    "gpt2": load_model("gpt2", device=device),
}

# Generate essays for one model at a time
for model_name, (pipeline_function, tokenizer) in essay_generation_models.items():
    logging.info(f"Starting essay generation for {model_name}")
    generated_essays = generate_and_save_essays(topics, model_name, pipeline_function, tokenizer)
    logging.info(f"Generated and saved essays for {model_name}")

    # Free GPU memory (if using GPU)
    if device.type == 'cuda':
        torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda


Load Essays

In [ ]:
def load_essays_from_files(model_name, topic_ids):
    essays = []
    for topic_id in topic_ids:
        filename = f"essay_{model_name}_{topic_id}.txt"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                essays.append((topic_id, f.read()))
        else:
            logging.warning(f"Essay file not found: {filename}")
    return essays

topics = [
    ("1097671", "Compare and contrast the importance of self-reliance and adaptability in healthcare."),
    ("1726150", "Evaluate the effectiveness of management consulting in addressing conflicts within marketing."),
    ("3211968", "Discuss the role of self-reliance in achieving success in software engineering.")
]

Scoring Functions

In [ ]:
import gc # Add this line to import the gc module

def scoring_model(essay, model, tokenizer, device):
    # Initialize a text classification pipeline using a fine-tuned model
    classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

    # Use the pipeline to classify the essay
    results = classifier(essay)

    # Ensure results is a list of dictionaries
    if isinstance(results, list) and len(results) > 0 and isinstance(results[0], dict):
        score = next((r['score'] for r in results if r['label'] == 'POSITIVE'), np.random.rand())
    else:
        # If results is not in the expected format, log a warning and return a random score
        logging.warning("Unexpected results format. Returning random score.")
        score = np.random.rand()

    if isinstance(score, np.ndarray):
        score = score.item() if score.size == 1 else score[0]

    del classifier  # Delete the classifier to free up memory
    gc.collect()    # Explicitly call garbage collection

    return score * 9



def get_llm_score(essay, model_name, essay_generation_models):
    """Gets the score for an essay from a specific model."""

    # Get the model and tokenizer from pre-loaded models
    result = essay_generation_models.get(model_name)

    # Check if the model was loaded successfully
    if result is None:
        logging.error(f"Model {model_name} not loaded or loading failed.")
        return 0  # or any default score you want to assign

    model, tokenizer = result # Now unpack the result

    # Now just the model variable represents the model part of the pipeline
    model = model.model

    # Re-initialize model to its original state (without the copy)
    model = model.to(device)

    # Call scoring_model to get the score
    score = scoring_model(essay, model, tokenizer, device)

    # Optional: Delete the model to free memory immediately after use
    del model
    gc.collect()
    torch.cuda.empty_cache()

    return score



def evaluate_essays_with_llms(essays, essay_models, essay_generation_models):
    all_scores = []
    judge_scores = {model_name: [] for model_name in essay_models}
    for topic_id, essay in essays:
        scores = {model_name: get_llm_score(essay, model_name, essay_generation_models) for model_name in essay_models}
        for model_name, score in scores.items():
            judge_scores[model_name].append(score)
        all_scores.append(scores)
    return all_scores, judge_scores


def calculate_metrics(all_scores, judge_scores):
    avg_q = np.mean([np.mean(list(scores.values())) for scores in all_scores])
    avg_h = np.mean([np.std(list(scores.values())) for scores in all_scores])
    min_v = min([np.var(scores) for scores in judge_scores.values()])
    return avg_q, avg_h, min_v

def calculate_english_confidence(essay, device='cpu'):  # Default to CPU
    try:
        # Explicitly move model to the specified device
        model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2", device=device)

        embedding = model.encode(essay, convert_to_tensor=True)
        ref_embedding = model.encode("This is a grammatically correct and fluent sentence.", convert_to_tensor=True)

        # Move embeddings to CPU before calculating similarity to free GPU memory
        similarity = cosine_similarity([embedding.cpu().numpy()], [ref_embedding.cpu().numpy()])[0][0]

        del model, embedding, ref_embedding  # Delete to free memory
        gc.collect()
        torch.cuda.empty_cache()  # Empty CUDA cache if using GPU

        return similarity
    except RuntimeError as e:
        if "out of memory" in str(e):
            logging.warning(f"CUDA out of memory in calculate_english_confidence, switching to CPU. Error: {e}")
            # If still out of memory, consider smaller models or batch processing
            return 0.5  # Or a reasonable default value
        else:
            logging.error(f"Runtime error in calculate_english_confidence: {e}")
            raise e

def calculate_sequence_similarity(essays):
    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    embeddings = model.encode([essay[1] for essay in essays], convert_to_tensor=True)
    similarity_matrix = cosine_similarity(embeddings.cpu().numpy())
    return np.mean(similarity_matrix)

def self_evaluate(essay, judge_scores, avg_e, avg_s, avg_q):
    # Get LLM-based scores
    essay_scores = np.array(judge_scores)

    # Metrics to consider in self-evaluation
    english_confidence = avg_e
    sequence_similarity = avg_s
    quality_score = avg_q

    # Weights for the different factors based on importance
    weight_llm = 0.5  # Weight of LLM scores in evaluation
    weight_english = 0.2  # Weight of English confidence score
    weight_similarity = 0.2  # Weight of sequence similarity
    weight_quality = 0.1  # Weight of overall quality score

    # Combining all factors for the reward function
    reward = (weight_llm * np.mean(essay_scores)) + \
             (weight_english * english_confidence) + \
             (weight_similarity * sequence_similarity) + \
             (weight_quality * quality_score)
    reward = np.clip(reward, 0, 1)

    return reward

Essay Scoring

In [ ]:
def main():
    if device.type == 'cuda':
        torch.cuda.empty_cache()

    # Define the pre-loaded essay models for evaluation
    essay_models = {
        "gpt-neo": "EleutherAI/gpt-neo-1.3B",
        "t5": "google/flan-t5-large",
        "gpt2": "gpt2",
    }

    # Load models for evaluation
    essay_generation_models = {}
    for model_name, model_path in essay_models.items():
        try:
            essay_generation_models[model_name] = load_model(model_path, device)
        except RuntimeError as e:
            logging.warning(f"Failed to load {model_name} due to memory constraints: {e}")
            essay_generation_models[model_name] = None

    # Define the topics associated with essays
    topics = [
        ("1097671", "Compare and contrast the importance of self-reliance and adaptability in healthcare."),
        ("1726150", "Evaluate the effectiveness of management consulting in addressing conflicts within marketing."),
        ("3211968", "Discuss the role of self-reliance in achieving success in software engineering.")
    ]

    # Load the essays from files (make sure the essays have already been generated and stored)
    generated_essays = []
    for model_name in essay_models.keys():
        generated_essays.extend(load_essays_from_files(model_name, [topic[0] for topic in topics]))

    # Ensure there are essays to evaluate
    if not generated_essays:
        logging.error("No essays found for evaluation.")
        return

    # Evaluate essays with LLMs
    all_scores, judge_scores = evaluate_essays_with_llms(generated_essays, essay_models, essay_generation_models)

    # Calculate evaluation metrics
    avg_q, avg_h, min_v = calculate_metrics(all_scores, judge_scores)

    # Calculate additional metrics like English confidence (avg_e) and sequence similarity (avg_s)
    avg_e = np.mean([calculate_english_confidence(essay[1]) for essay in generated_essays])
    avg_s = calculate_sequence_similarity(generated_essays)
    avg_s = max(avg_s, 0.2)  # Ensure sequence similarity doesn't go below 0.2

    # Compute the final score
    if avg_q < 9:  # Avoid division by zero
        final_score = (avg_h * min_v * avg_e) / (avg_s * (9 - avg_q))
        print(f"Final Score: {final_score:.2f}")
    else:
        logging.error("Invalid avg_q value leading to division by zero.")

# Call main function to run the evaluation
main()


Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda
Device set to use cuda


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Final Score: 0.24


Model Training Loop

In [ ]:
import time
import torch
import logging
import os
import numpy as np
from transformers import AdamW  # Import AdamW optimizer

# Set up logging for debugging and info
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def train_model(model, generated_essays, all_scores, judge_scores):
    # Access the underlying model for training
    model = model.model

    # Assuming model is a Hugging Face model
    optimizer = AdamW(model.parameters(), lr=5e-5)  # Adjust learning rate as needed

    # Training loop logic (simplified for demonstration)
    for essay, scores in zip(generated_essays, all_scores):
        # ... (process essay and scores to calculate loss)
        # Example: Assume you have a function 'calculate_loss'
        loss = calculate_loss(essay, scores)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Define the main function
def main():
    if device.type == 'cuda':
        torch.cuda.empty_cache()

    # Define the pre-loaded essay models for evaluation
    essay_models = {
        "gpt-neo": "EleutherAI/gpt-neo-1.3B",
        "t5": "google/flan-t5-large",
        "gpt2": "gpt2",
    }

    # Load models for evaluation
    essay_generation_models = {}
    for model_name, model_path in essay_models.items():
        try:
            essay_generation_models[model_name] = load_model(model_path, device)
        except RuntimeError as e:
            logging.warning(f"Failed to load {model_name} due to memory constraints: {e}")
            essay_generation_models[model_name] = None

    # Define the topics associated with essays
    topics = [
        ("1097671", "Compare and contrast the importance of self-reliance and adaptability in healthcare."),
        ("1726150", "Evaluate the effectiveness of management consulting in addressing conflicts within marketing."),
        ("3211968", "Discuss the role of self-reliance in achieving success in software engineering.")
    ]

    num_training_epochs = 5  # Adjust as needed
    best_score = 0  # Initialize best score for early stopping
    patience = 3  # Number of epochs to wait before stopping
    epochs_without_improvement = 0

    for epoch in range(num_training_epochs):
        # Load the essays from files
        generated_essays = []
        for model_name in essay_models.keys():
            generated_essays.extend(load_essays_from_files(model_name, [topic[0] for topic in topics]))

        # Ensure there are essays to evaluate
        if not generated_essays:
            logging.error("No essays found for evaluation.")
            return

        # Evaluate essays with LLMs
        all_scores, judge_scores = evaluate_essays_with_llms(generated_essays, essay_models, essay_generation_models)

        # Calculate evaluation metrics
        avg_q, avg_h, min_v = calculate_metrics(all_scores, judge_scores)

        # Calculate additional metrics
        avg_e = np.mean([calculate_english_confidence(essay[1]) for essay in generated_essays])
        avg_s = calculate_sequence_similarity(generated_essays)
        avg_s = max(avg_s, 0.2)  # Ensure sequence similarity doesn't go below 0.2

        # Compute the final score
        if avg_q < 9:  # Avoid division by zero
            final_score = (avg_h * min_v * avg_e) / (avg_s * (9 - avg_q))
            print(f"Final Score: {final_score:.2f}")
        else:
            logging.error("Invalid avg_q value leading to division by zero.")
            final_score = 0 # Set to 0 to allow for early stopping check

        # Early stopping check
        if final_score > best_score:
            best_score = final_score
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            logging.info("Early stopping triggered.")
            break # Exit training loop

        # Update model weights
        for model_name, (model, tokenizer) in essay_generation_models.items():
            if model is not None:
                logging.info(f"Training {model_name}")
                train_model(model, generated_essays, all_scores, judge_scores)  # Call your implemented train_model function
                # Save the updated model weights
                model.save_pretrained(f"./fine_tuned_{model_name}")
                logging.info(f"Saved fine-tuned {model_name} weights")
            else:
                logging.warning(f"Model {model_name} not loaded. Skipping training.")

# Call main function to run the evaluation
main()

NameError: name 'device' is not defined

In [ ]:
# After training, save the fine-tuned models
for i, model in enumerate(models):
    model.save_pretrained(f"./fine_tuned_{model_names[i]}")

print("Training and Testing Complete!")

Submission

In [ ]:
import pandas as pd

# Function to prepare and save essays for submission
def create_submission_file(essays, scores, filename="submission.csv"):
    rows = []

    essay_id = 1097671  # Starting ID, you can change this or make it dynamic

    for i, model_essays in essays.items():
        for essay in model_essays:
            row = {
                'id': essay_id,
                'essay': essay
            }
            rows.append(row)
            essay_id += 1  # Increment the ID for each new essay

    # Save to CSV
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Submission saved to {filename}")

# Example essays (generated previously)
essays = {
    "mistral": ["AI is both a blessing and a curse for humanity.", "The impact of AI is vast and varied."],
    "t5": ["AI can change the world, but it also poses risks.", "Technological advancement is a double-edged sword."],
    "gpt2": ["Artificial intelligence is shaping the future.", "AI presents both opportunities and challenges."]
}

# Call the function to create a submission file
create_submission_file(essays, scores)